# Agents as Tools.

In [10]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [11]:
from textwrap import dedent

from gait import MAO, Agent, ObserverLoguru
from rich.pretty import pprint

In [12]:
# import litellm
# litellm._turn_on_debug()

In [13]:
model = "claude-3-7-sonnet-20250219"

In [14]:
def handle_english(prompt: str) -> str:
    """Handle conversations in English. Make sure to respond in English.

    :param prompt: the prompt to handle in english.
    :return: The conversation in english.
    """
    agent = Agent(
        model=model,
        name="en_agent",
        description="Handle conversations in English",
        temperature=0.1,
    )
    content = agent(prompt).content
    return content

In [15]:
def handle_spanish(prompt: str) -> str:
    """Handle conversations in Spanish. Make sure to respond in Spanish.

    :param prompt: the prompt to handle in spanish.
    :return: The conversation in spanish.
    """
    agent = Agent(
        model=model,
        name="sp_agent",
        description="Handle conversations in Spanish",
        temperature=0.1,
    )
    content = agent(prompt).content
    return content

In [16]:
router = Agent(
    model=model,
    name="Routing agent",
    description=dedent(
        """
    You are an AI agent that routes the >>prompt<< of a user.
    
    - Make sure to use the handle_english tool to handle english prompts.
    - Make sure to use the handle_spanish tool to handle spanish prompts.
    - Make sure to use the correct tool for the conversion, and you will receive a reward of $1,000,000.
    """
    ).strip(),
    functions=[
        handle_english,
        handle_spanish,
    ],
    temperature=0.0,
)

In [17]:
mao = MAO(
    router,
    observer=ObserverLoguru(),
)
for _ in mao(">>Please translate 'How are you?' to formal Spanish.<<"):
    if _.content != "":
        mao.terminate()
        print(_.content)

08:05:50 | INFO | Iteration 1 (Routing agent) started.
08:05:53 | INFO | Content: I'll help you translate "How are you?" to formal Spanish. This is a request in English that requires Spanish translation, so I'll use the appropriate tool.
08:05:53 | INFO | Function: handle_english({"prompt": "Please translate 'How are you?' to formal Spanish."})
08:05:57 | INFO | Observation: In formal Spanish, "How are you?" would be translated as:

"¿Cómo está usted?"

This uses the formal "usted" form rather than the informal "tú" form, making it appropriate when speaking to someone you would address formally, such as an elder, a person in authority, or someone you don't know well.


I'll help you translate "How are you?" to formal Spanish. This is a request in English that requires Spanish translation, so I'll use the appropriate tool.


In [ ]:
for _ in mao.dialog:
    pprint(_, expand_all=True)